In [3]:
import os

In [4]:
%pwd

'c:\\projects\\sms_spam\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\projects\\sms_spam'

In [7]:
import pandas as pd

In [8]:
df=pd.read_csv("artifacts/data_ingestion/spam_classifier.csv")
df.head()

,Target,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Target  5572 non-null   object
 1   Text    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [10]:
df.isnull().sum()

Target    0
Text      0
dtype: int64

In [11]:
df.shape

(5572, 2)

In [12]:
df.columns

Index(['Target', 'Text'], dtype='object')

In [17]:
import os

In [19]:
from src.spam_classifier import logger